## Task allocation in a multiprocessor system

In [2]:
import numpy as np
import random

from ortools.sat.python import cp_model

rng = np.random.default_rng(0)
random.seed(0)

In [3]:
def generate_instance(N=100, speeds=(1.0, 1.25, 1.5, 1.75), low=10.0, high=90.0, seed=None):
    r = np.random.default_rng(seed)
    base = r.uniform(low, high, size=N)
    speeds = np.asarray(speeds, dtype=float)
    return base, speeds

def makespan(assignment, base_times, speeds):
    M = len(speeds)
    loads = np.zeros(M, dtype=float)
    # Time on processor p is base/speed[p]
    times = base_times / speeds[np.asarray(assignment)]
    np.add.at(loads, assignment, times)
    return float(loads.max())

def loads_vector(assignment, base_times, speeds):
    M = len(speeds)
    loads = np.zeros(M, dtype=float)
    times = base_times / speeds[np.asarray(assignment)]
    np.add.at(loads, assignment, times)
    return loads


In [4]:
def random_individual(N, M, r=None):
    r = np.random.default_rng() if r is None else r
    return r.integers(0, M, size=N, dtype=np.int32)

def tournament_select(pop, fitness, tsize, r):
    idx = r.integers(0, len(pop), size=tsize)
    best = idx[0]
    for j in idx[1:]:
        if fitness[j] < fitness[best]:
            best = j
    return best

def one_point_crossover(a, b, r):
    n = len(a)
    if n < 2:
        return a.copy(), b.copy()
    cut = int(r.integers(1, n))
    c1 = np.concatenate([a[:cut], b[cut:]])
    c2 = np.concatenate([b[:cut], a[cut:]])
    return c1, c2

def mutate(ind, M, pm, r):
    child = ind.copy()
    mask = r.random(len(child)) < pm
    if mask.any():
        child[mask] = r.integers(0, M, size=int(mask.sum()))
    return child


In [5]:
def genetic_task_allocation(base_times,
    speeds,
    pop_size=60,
    generations=400,
    tournament_size=3,
    crossover_prob=0.9,
    mutation_prob=0.02,
    elitism=1,
    seed=None,
):
    r = np.random.default_rng(seed)
    N = len(base_times)
    M = len(speeds)

    pop = [random_individual(N, M, r) for _ in range(pop_size)]
    fit = np.array([makespan(ind, base_times, speeds) for ind in pop], dtype=float)

    best_idx = int(fit.argmin())
    best = pop[best_idx].copy()
    best_fit = float(fit[best_idx])

    for _ in range(generations):
        elite_idx = np.argsort(fit)[:elitism]
        new_pop = [pop[i].copy() for i in elite_idx]

        while len(new_pop) < pop_size:
            p1 = pop[tournament_select(pop, fit, tournament_size, r)]
            p2 = pop[tournament_select(pop, fit, tournament_size, r)]
            
            if r.random() < crossover_prob:
                c1, c2 = one_point_crossover(p1, p2, r)
            else:
                c1, c2 = p1.copy(), p2.copy()
                
            c1 = mutate(c1, M, mutation_prob, r)
            c2 = mutate(c2, M, mutation_prob, r)
            new_pop.append(c1)
            
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        pop = new_pop
        fit = np.array([makespan(ind, base_times, speeds) for ind in pop], dtype=float)

        gen_best_idx = int(fit.argmin())
        if fit[gen_best_idx] < best_fit:
            best_fit = float(fit[gen_best_idx])
            best = pop[gen_best_idx].copy()

    return best, best_fit


In [6]:
def run_multiple_times(base_times, speeds, runs=10, seed=0, **ga_kwargs):
    best_sol = None
    best_cost = float('inf')
    for i in range(runs):
        sol, cost = genetic_task_allocation(base_times, speeds, seed=seed + i, **ga_kwargs)
        if cost < best_cost:
            best_sol, best_cost = sol, cost
    return best_sol, best_cost


In [7]:
base_times, speeds = generate_instance(N=100, speeds=(1.0, 1.25, 1.5, 1.75), seed=42)

best_assign, best_dt = run_multiple_times(
    base_times,
    speeds,
    runs=10,
    pop_size=80,
    generations=500,
    mutation_prob=0.02,
    crossover_prob=0.9,
    tournament_size=3,
    elitism=2,
)

print('Best delta t:', best_dt)
print('Loads per processor:', loads_vector(best_assign, base_times, speeds))
print('Tasks per processor:', np.bincount(best_assign, minlength=len(speeds)))


Best delta t: 890.0326211632515
Loads per processor: [889.77585725 890.03262116 889.70707122 889.6402432 ]
Tasks per processor: [22 25 26 27]


In [12]:
import numpy as np
from ortools.sat.python import cp_model

def solve_allocation_cpsat(base, speeds, time_limit_s=10, scale=1000, workers=2):
    base = np.asarray(base, dtype=float)
    speeds = np.asarray(speeds, dtype=float)

    N = base.shape[0]
    M = speeds.shape[0]
    p = np.rint((base[:, None] / speeds[None, :]) * scale).astype(np.int64)

    model = cp_model.CpModel()
    x = [[model.NewBoolVar(f"x_{i}_{k}") for k in range(M)] for i in range(N)]
    for i in range(N):
        model.AddExactlyOne(x[i])
    load_ubs = [int(p[:, k].sum()) for k in range(M)]
    loads = [model.NewIntVar(0, load_ubs[k], f"load_{k}") for k in range(M)]
    for k in range(M):
        model.Add(loads[k] == sum(x[i][k] * int(p[i, k]) for i in range(N)))

    T_ub = max(load_ubs)
    T = model.NewIntVar(0, int(T_ub), "T")
    model.AddMaxEquality(T, loads)
    model.Minimize(T)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = float(time_limit_s)
    solver.parameters.num_search_workers = int(workers)

    status = solver.Solve(model)
    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return None

    assignment = np.empty(N, dtype=int)
    for i in range(N):
        for k in range(M):
            if solver.Value(x[i][k]):
                assignment[i] = k
                break
    loads_val = np.array([solver.Value(v) for v in loads], dtype=float) / scale
    makespan = solver.Value(T) / scale

    return {
        "assignment": assignment,
        "loads": loads_val,
        "makespan": makespan,
        "status": "OPTIMAL" if status == cp_model.OPTIMAL else "FEASIBLE",
    }


In [11]:
res = solve_allocation_cpsat(base_times, speeds, time_limit_s=30)
print(res["status"])
print("makespan:", res["makespan"])
print("loads:", res["loads"])

FEASIBLE
makespan: 889.777
loads: [889.766 889.777 889.767 889.764]
